## Import Modules

In [1]:
import pdal
import sys
import os
import json
sys.path.append("../../../PythonScripts")
from glob import glob
import open3d as o3d
import os
import numpy as np
import pandas as pd
from multiprocessing import Pool
from pipeline_functions import downscale_las, get_scales, downscale_ply, getFeatures,getFeaturesParallel
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from time import time
# from scipy.spatial import cKDTree
from sklearn.neighbors import KDTree, BallTree
# from scipy.spatial import KDTree
import pickle
from datetime import datetime, timezone
from itertools import repeat # Used for starmap function
import gc # Used to free up memory to prevent kernel restarting

sns.set()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Get number of processors to run loop

In [2]:
processors = os.cpu_count() - 8

## Get File to run

In [3]:
# ROOT = """/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117.las"""
ROOT = """/home/sspiegel/CapstoneData/Paris/training_10_classes/Lille1_1.ply"""
# ROOT = """/home/sspiegel/CapstoneData/PA/225019325.las"""

## Load in base point cloud

In [4]:
pipeline_json = [
    {
        "type": "readers.ply",
        "filename": ROOT
    }
        
]


pipeline = pdal.Pipeline(json.dumps(pipeline_json))

# Execute the pipeline
# This will process the data according to the stages defined in the pipeline
pipeline.execute()

xy = pipeline.arrays[0]

mta = pipeline.metadata



In [5]:
xyzT = np.array((xy["X"], xy["Y"], xy["Z"])).T

In [6]:
xyz = xyzT - np.min(xyzT, axis = 0)

In [7]:
# xyz

## Get Classification

In [8]:
cls = xy["Classification"].astype(int)

In [9]:
scls = get_scales(S=6) # Use base paprameters from function

## Instantiate point cloud

In [10]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)

In [11]:
pc_dict = []
for s in scls:
    
    pc = pcd.voxel_down_sample(s[1])
    # pc_array = np.asarray(pc.points)
    pc_dict.append({
      "r" : s[0],
     "grid_size" : s[1],
     "point_cloud" : pc
     # "point_cloud_array" : pc_carray
    })


## Get scaling factors

### Downscale point clouds: Methodology

* Downscale point clouds based on multiscaling factor
* The scaling factor chosen from [Thomas Hugues et. al](https://ieeexplore.ieee.org/document/8490990/)
* Parameters
  * initial radius ($r_0$): 0.1
  * Number of scales ($S$): 6
  * Base exponent for expanding radius ($\gamma$): 2
  * scaling factor of grid ($\rho$): 5
  * radius of sphere for scale $s$: $r_s = r_0 * \gamma^{s}$
  * downsampled voxel size: $\frac{r_s}{\rho}$

* Use multiprocessing to speed up process

### Sample point cloud (first radius)

In [12]:
test = pc_dict[2]

test_pc = test["point_cloud"]

# tree = BallTree(test_pc.points)
tree = KDTree(test_pc.points)

In [13]:
print(test)

{'r': 0.4, 'grid_size': 0.08, 'point_cloud': PointCloud with 6382529 points.}


In [14]:
test_pc_arr = np.asarray(test_pc.points)

## Get Distance matrix

In [15]:
distMat = tree.query_radius(pcd.points, r=test["r"])
# distMat = tree.query_ball_point(pcd.points, r=test["r"], workers = 1)


In [16]:
pcOut = np.asarray(pcd.points)


In [17]:
del pc_dict, xyzT, xyz, xy, pcd

gc.collect()

20

## Split and apply features algorithmKDTree

In [18]:
# DIV = 100000
# groups = distMat.shape[0] // DIV

In [19]:
s = time()

bigList = []

# for g in range(groups + 1):
#     print("""Processing batch %d...""" % (g+1))
#     practiceList = distMat[g*DIV:(g+1)*DIV]

args = zip(repeat(test_pc_arr), distMat)
with Pool(processes=processors) as pool:
    result = pool.starmap(getFeaturesParallel, args)
bigList = np.array(result)
    # bigList.append(result)
    # print(practiceList.shape)
    # break
    
e = time()

print("""\n\n\n\nDone!!!  Total time: %.4f seconds""" % (e - s))





Done!!!  Total time: 683.5492 seconds


In [20]:
# bigList = np.concatenate(bigList)

## Save as a pickle file

In [21]:
del distMat, tree

gc.collect()

0

In [22]:
# pickleOut = f"""/home/sspiegel/CapstoneData/Paris/Toronto_3D/pickleFiles/{datetime.now(timezone.utc).strftime("%Y_%m_%dT%H_%M")}_L001_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}.pkl"""

In [23]:
getTime = datetime.now(timezone.utc).strftime("%Y_%m_%dT%H_%M")

featureOut = f"""/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/{getTime}_Lille1_1_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_features.npz"""
# pickleOutPC = f"""/home/sspiegel/CapstoneData/Paris/Toronto_3D/pickleFiles/{getTime}_L001_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_pointcloud.pkl"""

# outDict = { "features" : bigList, "classification" : cls}
# # outDict = {"PointCloud" : pcOut}


# with open(pickleOut, 'wb') as f:
#     pickle.dump(outDict, f)

np.savez(featureOut,array1 = pcOut, array2 = bigList, array3 = cls)

## Save stats

In [24]:
sts = f"""/home/sspiegel/CapstoneData/Paris/training_10_classes/stats/{datetime.now(timezone.utc).strftime("%Y_%m_%d")}_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_features.json"""


In [25]:
jsStats = {
    "filename" : ROOT,
    "radius" : test["r"],
    "grid_size" : test["grid_size"],
    "total_time" : "%.4f seconds" % (e - s)
}

In [26]:
with open(sts, "w") as jsOut:
    jsOut.write(json.dumps(jsStats))
